<a href="https://colab.research.google.com/github/rpizarrog/innovacion-empresarial/blob/main/notebook_python/Caso_18_Validaci%C3%B3n_cruzada_Da%C3%B1os_al_coraz%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivo

Realizar proceso de validación cruzada para evaluar modelos de clasificación con datos relacionados con la predicción de daños al corazón



# Descripcón

## Los datos

Las variables de interés

Las variables de entrada o variables independientes:

* 1. "BMI": Indice de masa corporal con valores entre 12.02 y 94.85.

* 2. "Smoking": Si la persona es fumadora o no con valores categóritos de 'Yes' o 'No'. Ya con datos previamente transformados: [1 Yes | 2 No]

* 3. "AlcoholDrinking" : Si consume alcohol o no, con valores categóricos de 'Yes' o 'No'. Ya con datos previamente transformados: [1 Yes | 2 No]

* 4. "Stroke": Si padece alguna anomalía cerebrovascular, apoplejia o algo similar, con valores categóricos de 'Yes' o 'No'. Ya con datos previamente transformados: [1 Yes | 2 No]

* 5. "PhysicalHealth" Estado físico en lo general con valores entre 0 y 30.

* 6. "MentalHealth". Estado mental en lo general con valores entre 0 y 30.

* 7. "DiffWalking" . Que si se le dificulta caminar o tiene algún padecimiento al caminar, con valores categóritoc de 'Yes' o 'No'. Ya con datos previamente transformados: [1 Yes | 2 No]

* 8. "Sex": Género de la persona, con valores de 'Female' y 'Male' para distinguir al género femenino y masculino respectivamente. Ya con datos previamente transformados: [1 Female | 2 Male]

* 9. "AgeCategory": Una clasificación de la edad de la persona de entre 18 y 80 años. La primera categoría con un rango de edad entre 18-24, a partir de 25 con rangos de 5 en 5 hasta la clase de 75-80 y una última categoría mayores de 80 años. Ya con datos previamente transformados: [1 - 13].

* 10. "Race". Raza u origen de la persona con valores categóricos de 'American Indian/Alaskan Native', 'Asian','Black', 'Hispanic', 'Other' y'White'..Ya con datos previamente transformados: [1 - 6]

* 11. "Diabetic". Si padece o ha padecido de diabetes en cuatro condiciones siendo Yes y No para si o no: 'No', 'borderline diabetes' condición antes de detectarse diabetes tipo 2, 'Yes', y 'Yes (during pregnancy)' durante embarazo. Ya con datos previamente transformados: [1 - 4]

* 12. "PhysicalActivity" que si realiza actividad física, con valores categóricos de 'Yes' o 'No'. Ya con datos previamente transformados: [1 Yes | 2 No]

* 13. "GenHealth": EStado general de salud de la persona con valores categóricos de 'Excellent', 'Very good', 'Good', 'Fair' y 'Poor' con significado en español de excelente, muy buena, buena, regular y pobre o deficiente. Ya con datos previamente transformados: [1 - 5]

* 14. "SleepTime": valor numérico de las horas de sueño u horas que duerme la persona con valores en un rango entre 1 y 24.

* 15. "Asthma": si padece de asma o no, con valores categóricos de 'Yes' o 'No'. Ya con datos previamente transformados: [1 Yes | 2 No]

* 16. "KidneyDisease": si tiene algún padecimiento en los riñones, con valores categóricos de 'Yes' o 'No'. Ya con datos previamente transformados: [1 Yes | 2 No]

* 17. "SkinCancer": si padece algún tipo de cáncer de piel, con valores categóricos de 'Yes' o 'No'. Ya con datos previamente transformados: [1 Yes | 2 No]

La variable de interés como dependiente o variable de salida es la de daño al corazón (HeartDisease), con valores categóricos de *1 = 'Yes' o 0 = 'No'.* [1 Yes | 0 No]

* 18. HeartDisease

Son 319795 observaciones y 18 variables

# Desarrollo



## Cargar librerías


In [4]:
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output


import time # Para medir el tiempo de ejecución

import matplotlib.pyplot as plt


## Cargar funciones



In [5]:
def f_diagrama_barra(datos):
    conteo = datos['HeartDisease'].value_counts()
    colores = ['blue', 'red']  # Colores para las barras

    # Crear el gráfico de barras con colores específicos
    plt.figure(figsize=(8, 6))
    barras = plt.bar(conteo.index, conteo.values, color=colores)

    # Añadir etiquetas con la frecuencia en cada barra
    for barra in barras:
        yval = barra.get_height()
        plt.text(barra.get_x() + barra.get_width()/2, yval, int(yval), ha='center', va='bottom')

    plt.title('Total de Daños al Corazón vs No Daños')
    plt.xlabel('Daño al Corazón (1 = Daño, 0 = No Daño)')
    plt.ylabel('Total de Casos')
    plt.xticks(ticks=[0, 1], labels=['No Daño', 'Daño'], rotation=0)
    plt.show()

## Cargar widgets


In [6]:
visualizar = widgets.Output()


# Función para actualizar y mostrar el contenido dinámicamente en el widget de salida
def f_visualizar_datos(datos1, datos2, titulo1, titulo2):
    with visualizar:
        #global titulo1, titulo2
        #titulo1 = "Primeros 20 registros de datos"
        visualizar.clear_output(wait=True)  # Limpiar el contenido anterior
        display(HTML(f"<h2>{titulo1}</h2>"))  # Establecer el nuevo título
        # Visualizar datos
        display(datos1)

        #titulo2 = "Últimos 20 registros de datos"
        display(HTML(f"<h2>{titulo2}</h2>"))  # Establecer el nuevo título
        # Visualizar datos
        display(datos2)

## Cargar datos



In [7]:
# Carga el archivo CSV
ruta = 'https://raw.githubusercontent.com/rpizarrog/innovacion-empresarial/main/datos/danios%20al%20corazon%20numericos%20limpios01.csv'  # Actualiza con la ruta correcta
datos = pd.read_csv(ruta)

# Tamaño de la muestra deseado, por ejemplo, 1000 registros
n = 566
muestra = datos.sample(n=n, random_state=2024)

# Llamar a la función para mostrar datos de entrenamiento ordenados por índice
f_visualizar_datos(muestra.head(20), muestra.tail(20), "Primeros registros de Muestra", "Últimos registros de Muestra")

# Para mostrar el widget de salida
display(visualizar)


Output()

## Variables independiente y dependiente

Se designa a la variable HeartDisease01 como la variable etiqueta y dependiente.

Las variables BMI, Smoking, AlcoholDrinking, Stroke, PhysicalHealth, MentalHealth, DiffWalking, Sex, AgeCategory, Race, Diabetic, PhysicalActivity, GenHealth, SleepTime, Asthma, KidneyDisease, SkinCancer son independientes.




In [8]:
# 'HeartDisease01' es la variable objetivo

# Divide los datos en características y variable objetivo
independientes = muestra.drop('HeartDisease01', axis=1)
dependiente = muestra['HeartDisease01']

## Los modelos


In [9]:


# Configura los modelos para la validación cruzada
modelos = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Linear SVM": SVC(kernel='linear'),
    "AdaBoost": AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=4), n_estimators=50),
}



In [ ]:
## Validación cruzada

In [18]:
from sklearn.model_selection import cross_validate, KFold
from sklearn.metrics import make_scorer, accuracy_score, recall_score, precision_score
from sklearn.metrics import confusion_matrix

# Función para calcular la especificidad
def specificity_score(y_true, y_pred):
    tn, fp, _, _ = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

# Marcar el tiempo de inicio
start_time = time.time()

# Configura la validación cruzada
val_cruzada = KFold(n_splits=5, shuffle=True, random_state=2024)

# Scorers personalizados
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, zero_division=0),
    'recall': make_scorer(recall_score, zero_division=0),  # Sensibilidad
    'specificity': make_scorer(specificity_score)
}

# Realiza la validación cruzada
estadisticos_modelos = {}
for modelo_nombre, model in modelos.items():
    results = cross_validate(model, independientes, dependiente, cv=val_cruzada, scoring=scoring)
    estadisticos_modelos[modelo_nombre] = {
        'accuracy_mean': results['test_accuracy'].mean(),
        'accuracy_std': results['test_accuracy'].std(),
        'precision_mean': results['test_precision'].mean(),
        'precision_std': results['test_precision'].std(),
        'recall_mean': results['test_recall'].mean(),
        'recall_std': results['test_recall'].std(),
        'specificity_mean': results['test_specificity'].mean(),
        'specificity_std': results['test_specificity'].std()
    }

# Marcar el tiempo de finalización
end_time = time.time()

# Calcular la duración restando el tiempo de inicio del tiempo de finalización
duration = end_time - start_time
print(f"El tiempo de ejecución fue de {duration} segundos.")

# Imprimir estadísticas de cada modelo
for modelo, estadisticas in estadisticos_modelos.items():
    print(f"{modelo}:")
    for stat, value in estadisticas.items():
        print(f"  {stat}: {value:.4f}")
    print()  # Añade una línea en blanco para separar cada modelo

El tiempo de ejecución fue de 2.2502927780151367 segundos.
Logistic Regression:
  accuracy_mean: 0.9188
  accuracy_std: 0.0169
  precision_mean: 0.4667
  precision_std: 0.4522
  recall_mean: 0.0922
  recall_std: 0.0920
  specificity_mean: 0.9885
  specificity_std: 0.0111

Decision Tree:
  accuracy_mean: 0.8852
  accuracy_std: 0.0215
  precision_mean: 0.2600
  precision_std: 0.0769
  recall_mean: 0.2317
  recall_std: 0.0854
  specificity_mean: 0.9407
  specificity_std: 0.0277

Random Forest:
  accuracy_mean: 0.9170
  accuracy_std: 0.0042
  precision_mean: 0.2000
  precision_std: 0.4000
  recall_mean: 0.0200
  recall_std: 0.0400
  specificity_mean: 0.9924
  specificity_std: 0.0071

Linear SVM:
  accuracy_mean: 0.9223
  accuracy_std: 0.0066
  precision_mean: 0.0000
  precision_std: 0.0000
  recall_mean: 0.0000
  recall_std: 0.0000
  specificity_mean: 1.0000
  specificity_std: 0.0000

AdaBoost:
  accuracy_mean: 0.9170
  accuracy_std: 0.0070
  precision_mean: 0.2500
  precision_std: 0.2236


# Interpretación

Se cargaron los datos con 319795 observaciones y 18 variables.

Se hizo un proceso de validación cruzada con los algoritmos: *Regresión logística, Árbol de clasificación, Random Forest, Máquinas de soporte vectorial lineal (SVM) y AdaBoost* con una muestra de 566 registros que es la raiz cuadrada de 319795.

El mejor algoritmo de clasificación tomando en cuenta el estadístico de *accuracy*, exactitud con los datos muestra fue el de *SVM de tipo lineal* con un valor aproximado de 92.23%.

Todos los algoritmos son altamente eficientes por encima del 90% con respecto a la exactitud.

Significado de los estadísticos:

* Accuracy o exactitud
$$
accuracy = \frac{VP + VN}{VP+FP+FN+VN}
$$

$$
n = \text{total de datos que participaron en el modelo = } {VP+FP+FN+VN}
$$

* La exactitud (o "accuracy") representa el porcentaje de predicciones correctas frente al total. Por tanto, es el cociente entre los casos bien clasificados por el modelo (verdaderos positivos y verdaderos negativos, es decir, los valores en la diagonal de la matriz de confusión), y la suma de todos los casos.

* Precision o precisión
$$
precision = \frac{VP}{VP + FP}
$$

* La precisión, (o“precision”) se refiere a lo cerca que está el resultado de una predicción del valor verdadero. Por tanto, es el cociente entre los casos positivos bien clasificados por el modelo y el total de predicciones positivas. Para calcularlo a mano, a partir de la matriz de confusión.

* Recall o recuperación
$$
recall = \frac{VP}{VP + FN}
$$

* La sensibilidad (o recall) representa la tasa de verdaderos positivos (True Positive Rate) ó TP. Es la proporción entre los casos positivos bien clasificados por el modelo, respecto al total de positivos.

* Especificity o especificidad (tasa de verdaderos negativos)

$$
especificity = \frac{VN}{VN + FP}
$$

* La especificidad, por su parte, es la tasa de verdaderos negativos, (“true negative rate”) o TN. Es la proporción entre los casos negativos bien clasificados por el modelo, respecto al total de negativos.

$$
\therefore \text{de dónde ... ...}
$$

* VP = Verdaderos Positivos. Le atinó
* FP = Falsos Positivos No acertó
* FN = Falsos Negativos. No acertó
* VN = Verdaderos Negativos. Le atinó.

Dentro del contexto de daños al corazón, ¿qué significa positivo y negativo?:
* positivo es que NO está enfermo del corazón *(HeartDisease='No')* y
* negativo es que SI está enfermo del corazón*(HeartDisease='Yes')*.

Y que significa verdadero y falso:

* Verdadero significa que la prediccón si le atina ya sea a lo positivo o al negativo
* False significa que la predicción no le atina ya sea positivo o negativo.

## Matriz:

$$
\begin{array}{c|c|c}
& \text{Predicción Positiva. No Daño} & \text{Predicción Negativa.Si Daño} \\
\hline
\text{Clase Positiva. No Daño} & VP & FN \\
\text{Clase Negativa. Si Daño} & FP & VN \\
\hline
\end{array}
$$

Al final del día ¿qué métrica elegir?
La conveniencia de usar una métrica otra como medida del estimador dependerá de cada caso en particular y, en concreto, del "costo" asociado a cada error de clasificación del algoritmo.

